In [14]:
from sklearn.model_selection import train_test_split
import pandas as pd
from pyfume.Clustering import Clusterer
from pyfume.EstimateAntecendentSet import AntecedentEstimator
from pyfume.EstimateConsequentParameters import ConsequentEstimator
from pyfume.SimpfulModelBuilder import SugenoFISBuilder
from pyfume.Tester import SugenoFISTester
from pandas import read_csv 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor


In [15]:
#DATA PREPROCESSING

#Load data
#data = read_csv('/Users/mariacarolina/Documents/IST/PYFUME_TUTORIAL/DATA/hairdryer.csv', header=None)

data = pd.read_csv('/Users/mariacarolina/Documents/IST/PYFUME_TUTORIAL/NN_SKLEARN/hairdryer.csv',header=None, names=["Voltage", "Temperature"]) 
var_names = data.columns.tolist()[:-1]



#X = data[["Voltage"]]  # X is a DataFrame
#y = data["Temperature"]  # y is a Series

#Convert dataframe to a numpy array
data = data.to_numpy()

# Separate features (X) and target (y)
X = data[:, 0]  # 'input voltage'
y = data[:, 1]  # 'temperature'

# Reshape X as it's a single feature
X = X.reshape(-1, 1)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X)


[[6.41]
 [3.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [6.41]
 [3.41]
 [3.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [6.41]
 [6.41]
 [6.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [6.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [6.41]
 [6.41]
 [6.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [3.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]
 [6.41]


In [16]:
# %% Train model
regr = MLPRegressor(hidden_layer_sizes=(20,20),random_state=42, max_iter=500)
regr.fit(X_train, y_train)

# %% Get model predictions
y_pred = regr.predict(X_test)

# %% Compute regression metrics
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error: {:.3f}".format(mse))
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error: {:.1f}%".format(mape*100))
exp_var = explained_variance_score(y_test, y_pred)
print("Explained Variance Score: {:.3f}".format(exp_var))

Mean Squared Error: 0.624
Mean Absolute Percentage Error: 14.6%
Explained Variance Score: 0.029


In [13]:
'''
#Cluster the input-output space
cl = Clusterer(x_train=X_train, y_train=y_train, nr_clus=10)
clust_centers, part_matrix, _ = cl.cluster(method='fcm')
print(f"Partition matrix shape: {part_matrix.shape}")  # Should be (n_samples, nr_clusters)

# Estimate membership functions parameters 
ae = AntecedentEstimator(X_train, part_matrix)
antecedent_params = ae.determineMF()
print(antecedent_params)

# Estimate consequent parameters
ce = ConsequentEstimator(X_train, y_train, part_matrix)
conseq_params = ce.suglms(global_fit=True, df=0.0)
print(conseq_params)

# %% Build first-order Takagi-Sugeno model
modbuilder = SugenoFISBuilder(
    antecedent_sets=antecedent_params, 
    consequent_parameters=conseq_params, 
    variable_names=var_names)

model = modbuilder.get_model()

# %% Get model predictions
modtester = SugenoFISTester(model, X_test, var_names)
y_pred = modtester.predict()[0]

# %% Compute regression metrics
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error: {:.3f}".format(mse))
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error: {:.1f}%".format(mape*100))
exp_var = explained_variance_score(y_test, y_pred)
print("Explained Variance Score: {:.3f}".format(exp_var))
'''



'\n#Cluster the input-output space\ncl = Clusterer(x_train=X_train, y_train=y_train, nr_clus=10)\nclust_centers, part_matrix, _ = cl.cluster(method=\'fcm\')\nprint(f"Partition matrix shape: {part_matrix.shape}")  # Should be (n_samples, nr_clusters)\n\n# Estimate membership functions parameters \nae = AntecedentEstimator(X_train, part_matrix)\nantecedent_params = ae.determineMF()\nprint(antecedent_params)\n\n# Estimate consequent parameters\nce = ConsequentEstimator(X_train, y_train, part_matrix)\nconseq_params = ce.suglms(global_fit=True, df=0.0)\nprint(conseq_params)\n\n# %% Build first-order Takagi-Sugeno model\nmodbuilder = SugenoFISBuilder(\n    antecedent_sets=antecedent_params, \n    consequent_parameters=conseq_params, \n    variable_names=var_names)\n\nmodel = modbuilder.get_model()\n\n# %% Get model predictions\nmodtester = SugenoFISTester(model, X_test, var_names)\ny_pred = modtester.predict()[0]\n\n# %% Compute regression metrics\nmse = mean_squared_error(y_test, y_pred)\np